# SQL Practice Problems using sqlite + Pandas + Jupyter notebooks and Northwinds Database 

###### DB: https://docs.microsoft.com/en-us/dotnet/framework/data/adonet/sql/linq/downloading-sample-databases



In [1]:
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect("Northwind_large.sqlite")

# check available tables 
import pandas as pd

cur = con.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
available_table=(cur.fetchall())


In [2]:
# Problem 32: High value customers: at least 1 order > 10000 in 2015

Q = '''Select
Customers.Id,
Customers.CompanyName,Orders.ID, 
sum(Quantity * UnitPrice) as TAM
From Customer Customers 
    Join "Order" Orders on Orders.CustomerID = Customers.ID 
    Join OrderDetail OrderDetails on Orders.ID= OrderDetails.OrderId 
    Where 
        OrderDate >= "20150101" and OrderDate < "20160101"
        
        Group By 
            Customers.ID,
            Customers.CompanyName,
            Orders.ID
            having TAM > 10000'''

pd.read_sql( Q , con)


,Id,CompanyName,Id,TAM
0,ALFKI,Alfreds Futterkiste,11565,37342.47
1,ALFKI,Alfreds Futterkiste,14496,41685.87
2,ALFKI,Alfreds Futterkiste,14606,21295.56
3,ALFKI,Alfreds Futterkiste,17114,37997.07
4,ALFKI,Alfreds Futterkiste,17765,26304.25
...,...,...,...,...
460,WOLZA,Wolski Zajazd,11670,35814.41
461,WOLZA,Wolski Zajazd,14449,34936.00
462,WOLZA,Wolski Zajazd,15391,42307.49
463,WOLZA,Wolski Zajazd,21074,36276.00


In [3]:
# Problem 33 Customers Ordering total > $15000 or more in 2015


Q = '''Select
Customers.Id,
Customers.CompanyName,
sum(Quantity *UnitPrice* (1-Discount)) as Amt_With_Discount
From Customer Customers 
    Join "Order" Orders on Orders.CustomerID = Customers.ID 
    Join OrderDetail OrderDetails on Orders.ID= OrderDetails.OrderId 
    Where 
        OrderDate >= "20150101" and OrderDate < "20160101"
        
        Group By 
            Customers.ID,
            Customers.CompanyName
            having Amt_With_Discount > 15000 order by 3'''

pd.read_sql( Q , con)

,Id,CompanyName,Amt_With_Discount
0,HUNGC,Hungry Coyote Import Store,28482.80
1,FOLIG,Folies gourmandes,46075.57
2,OLDWO,Old World Delicatessen,52038.56
3,SAVEA,Save-a-lot Markets,53756.33
4,RICSU,Richter Supermarkt,57696.80
...,...,...,...
86,ANTON,Antonio Moreno Taquería,300125.75
87,KOENE,Königlich Essen,318929.59
88,SUPRD,Suprêmes délices,375843.90
89,GOURL,Gourmet Lanchonetes,422673.23


In [4]:
# Problem 34 Orders placed Only on last day of month - we break the problem into two parts
# get last day of month, and show orders where date = last date of month

Q = "SELECT EmployeeId,Id, OrderDate from 'Order' where OrderDate = date(OrderDate,'start of month','+1 month','-1 day') ;"
pd.read_sql( Q , con)

,EmployeeId,Id,OrderDate
0,5,10269,2012-07-31
1,6,10317,2012-09-30
2,4,10343,2012-10-31
3,8,10399,2012-12-31
4,3,10432,2013-01-31
5,8,10460,2013-02-28
6,1,10461,2013-02-28
7,7,10490,2013-03-31
8,8,10491,2013-03-31
9,4,10522,2013-04-30


In [5]:
# Problem 35 for each employee, Orders placed Only on last day of month - we break the problem into two parts
# get last day of month, and show orders where date = last date of month

Q = "SELECT EmployeeId, count(*) from 'Order' where OrderDate = date(OrderDate,'start of month','+1 month','-1 day') group by 1;"
pd.read_sql( Q , con)

,EmployeeId,count(*)
0,1,2
1,2,4
2,3,4
3,4,8
4,5,1
5,6,1
6,7,1
7,8,4
8,9,1


In [6]:
# problem 36 Order ids with most items

Q = "SELECT OrderId, count(*) from OrderDetail group by 1 order by 2 desc limit 10;"
pd.read_sql( Q , con)

,OrderId,count(*)
0,27062,77
1,26911,77
2,26752,77
3,26569,77
4,26465,77
5,26444,77
6,26443,77
7,26433,77
8,26358,77
9,26324,77


In [7]:
# Excercise 38: Double entry item > 60

Q = "SELECT OrderId, Quantity, count(*) as counts from OrderDetail where Quantity >= 60 group by 1,2 having counts = 2;"
pd.read_sql( Q , con)

,OrderId,Quantity,counts
0,10263,60,2
1,10658,70,2
2,10990,65,2
3,11030,100,2


In [8]:
# Details of above Order

Q = '''
WITH proborder as (
select OrderId, Quantity, count(*) as counts from OrderDetail where Quantity >= 60 group by 1,2 having counts = 2
)

SELECT * from OrderDetail where OrderID in (
        select OrderID from proborder);'''
pd.read_sql( Q , con)

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10263/16,10263,16,13.90,60,0.25
1,10263/24,10263,24,3.60,28,0.00
2,10263/30,10263,30,20.70,60,0.25
3,10263/74,10263,74,8.00,36,0.25
4,10658/21,10658,21,10.00,60,0.00
5,10658/40,10658,40,18.40,70,0.05
6,10658/60,10658,60,34.00,55,0.05
7,10658/77,10658,77,13.00,70,0.05
8,10990/21,10990,21,10.00,65,0.00
9,10990/34,10990,34,14.00,60,0.15


In [9]:
#Excercise 41 Late Orders

Q = '''select * from "Order" where ShippedDate > RequiredDate;'''
pd.read_sql( Q , con)

,Id,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10264,FOLKO,6,2012-07-24,2012-08-21,2012-08-23,3,25.00,Folk och fä HB,Åkergatan 24,Bräcke,Northern Europe,S-844 67,Sweden
1,10271,SPLIR,6,2012-08-01,2012-08-29,2012-08-30,2,16.00,Split Rail Beer & Ale,P.O. Box 555,Lander,North America,82520,USA
2,10280,BERGS,2,2012-08-14,2012-09-11,2012-09-12,1,25.50,Berglunds snabbköp,Berguvsvägen 8,Luleå,Northern Europe,S-958 22,Sweden
3,10302,SUPRD,4,2012-09-10,2012-10-08,2012-10-09,2,30.00,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,Western Europe,B-6000,Belgium
4,10309,HUNGO,3,2012-09-19,2012-10-17,2012-10-23,1,28.75,Hungry Owl All-Night Grocers,8 Johnstown Road,Cork,British Isles,None,Ireland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,27040,SPLIR,2,2015-02-19 21:04:45,2015-03-15 19:52:16,2015-03-21 13:54:13,3,92.50,Split Rail Beer & Ale,P.O. Box 555,Lander,North America,82520,USA
3949,27042,OCEAN,2,2014-06-18 08:26:12,2014-06-18 19:28:10,2014-06-18 23:23:37,2,52.50,Trail's Head Gourmet Provisioners,722 DaVinci Blvd.,Kirkland,North America,98034,USA
3950,27055,SPLIR,9,2015-10-06 15:31:42,2015-10-13 16:23:06,2015-10-16 14:46:08,1,246.00,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany
3951,27056,FAMIA,7,2014-07-29 01:31:55,2014-08-01 18:55:28,2014-08-03 20:07:44,3,280.75,Ottilies Käseladen,Mehrheimerstr. 369,Köln,Western Europe,50739,Germany


In [10]:
#Excercise 42 Employee details of late orders


Q = '''
WITH late_orders as 
    (select * from "Order" where ShippedDate > RequiredDate)
    
    select EmployeeId, LastName, count(*) from late_orders a
    inner join Employee b on
    a.EmployeeId = b.Id
    
    group by 1,2
    order by 3
    
    
    
    ;'''
pd.read_sql( Q , con)

,EmployeeId,LastName,count(*)
0,3,Leverling,419
1,8,Callahan,421
2,2,Fuller,432
3,5,Buchanan,432
4,6,Suyama,437
5,7,King,438
6,1,Davolio,446
7,9,Dodsworth,451
8,4,Peacock,477


In [11]:
# Excercise 43,44,45 all orders vs total orders
Q = '''
WITH all_orders as 
    (select *, 
    CASE
        when ShippedDate > RequiredDate then 1 else 0 END as lateorder
    
    from "Order" )
    
    select EmployeeId, LastName, count(*) as AllOrders,sum(lateorder) as LATEORDERS from all_orders a
    inner join Employee b on
    a.EmployeeId = b.Id
    
    group by 1,2
    order by 3
    
    
    
    ;'''
pd.read_sql( Q , con)

,EmployeeId,LastName,AllOrders,LATEORDERS
0,2,Fuller,1805,432
1,9,Dodsworth,1835,451
2,7,King,1839,438
3,8,Callahan,1842,421
4,6,Suyama,1849,437
5,5,Buchanan,1859,432
6,4,Peacock,1907,477
7,1,Davolio,1918,446
8,3,Leverling,1964,419


In [23]:
# Excercise 46 % of late orders
Q = '''
WITH all_orders as 
    (select *, 
    CASE
        when ShippedDate > RequiredDate then 1 else 0 END as lateorder
    
    from "Order" )
    
    select EmployeeId, LastName, count(*) as AllOrders,sum(lateorder) as LATEORDERS,  sum(lateorder)*1.00/count(*) as pct_late from all_orders a
    inner join Employee b on
    a.EmployeeId = b.Id
    
    group by 1,2
    order by 3
    
    
    
    ;'''
pd.read_sql( Q , con)

,EmployeeId,LastName,AllOrders,LATEORDERS,pct_late
0,2,Fuller,1805,432,0.239335
1,9,Dodsworth,1835,451,0.245777
2,7,King,1839,438,0.238173
3,8,Callahan,1842,421,0.228556
4,6,Suyama,1849,437,0.236344
5,5,Buchanan,1859,432,0.232383
6,4,Peacock,1907,477,0.250131
7,1,Davolio,1918,446,0.232534
8,3,Leverling,1964,419,0.213340


In [24]:
Q = 'select * from "Order"'
pd.read_sql( Q , con)

,Id,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,16.75,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France
1,10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,22.25,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany
2,10250,HANAR,4,2012-07-08,2012-08-05,2012-07-12,2,25.00,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,South America,05454-876,Brazil
3,10251,VICTE,3,2012-07-08,2012-08-05,2012-07-15,1,20.25,Victuailles en stock,"2, rue du Commerce",Lyon,Western Europe,69004,France
4,10252,SUPRD,4,2012-07-09,2012-08-06,2012-07-11,2,36.25,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,Western Europe,B-6000,Belgium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16813,27061,FOLKO,5,2013-06-29 21:05:55,2013-08-02 04:10:53,2013-07-02 16:05:51,3,307.25,Familia Arquibaldo,"Rua Orós, 92",Sao Paulo,South America,05442-030,Brazil
16814,27062,FRANK,2,2014-01-19 12:27:11,2014-01-24 15:15:31,2014-01-27 02:14:31,2,550.50,Bon app',"12, rue des Bouchers",Marseille,Western Europe,13008,France
16815,27063,ALFKI,5,2014-10-15 09:51:09,2014-11-11 14:31:37,2014-10-16 06:26:55,1,328.50,Furia Bacalhau e Frutos do Mar,Jardim das rosas n. 32,Lisboa,Southern Europe,1675,Portugal
16816,27064,TRADH,8,2013-02-07 02:06:05,2013-03-14 09:43:16,2013-02-24 10:15:47,3,357.00,Wilman Kala,Keskuskatu 45,Helsinki,Scandinavia,21240,Finland


In [29]:
# Excercise 55 First Order in each country

Q = '''
with temp_t as (
select *,  Row_Number() over (Partition by ShipCountry Order by OrderDate) as rankorder_c 

from "Order"
    
   order by  ShipCountry, OrderDate
)

select * from temp_t where rankorder_c = 1

    ;'''
pd.read_sql( Q , con)

,Id,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry,rankorder_c
0,23304,RANCH,8,2012-07-10 11:12:43,2012-07-12 00:56:52,2012-07-11 08:48:25,3,219.50,Cactus Comidas para llevar,Cerrito 333,Buenos Aires,South America,1010,Argentina,1
1,24940,CONSH,4,2012-07-10 02:35:32,2012-07-21 23:48:44,2012-07-11 13:59:17,2,71.50,Ernst Handel,Kirchgasse 6,Graz,Western Europe,8010,Austria,1
2,10252,SUPRD,4,2012-07-09,2012-08-06,2012-07-11,2,36.25,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,Western Europe,B-6000,Belgium,1
3,10250,HANAR,4,2012-07-08,2012-08-05,2012-07-12,2,25.00,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,South America,05454-876,Brazil,1
4,19699,QUICK,1,2012-07-10 06:37:47,2012-07-14 19:24:42,2012-07-16 02:05:12,1,187.75,Mère Paillarde,43 rue St. Laurent,Montréal,North America,H1J 1C3,Canada,1
5,22117,TRADH,9,2012-07-12 21:59:26,2012-07-13 06:42:36,2012-07-19 15:50:37,2,175.00,Simons bistro,Vinbæltet 34,Kobenhavn,Northern Europe,1734,Denmark,1
6,15044,PRINI,5,2012-07-18 08:03:21,2012-07-24 09:53:42,2012-08-05 07:27:33,2,335.00,Wartian Herkku,Torikatu 38,Oulu,Scandinavia,90110,Finland,1
7,10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,16.75,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France,1
8,10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,22.25,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany,1
9,18757,QUICK,5,2012-07-21 18:59:04,2012-07-31 21:27:02,2012-08-10 06:44:46,1,276.50,Hungry Owl All-Night Grocers,8 Johnstown Road,Cork,British Isles,None,Ireland,1


In [39]:
# 56/57 Customers with multiple orders in 5 days

Q = '''
with temp_t as (
select CustomerId,Id,OrderDate,
    Lead(OrderDate,1) over(Partition by CustomerId order by CustomerId,OrderDate) as next_order

    from "Order"
order by CustomerID, OrderDate
)

select * from temp_t

    ;'''
pd.read_sql( Q , con)

,CustomerId,Id,OrderDate,next_order
0,ALFKI,19127,2012-07-11 18:10:00,2012-07-15 15:57:34
1,ALFKI,25934,2012-07-15 15:57:34,2012-07-19 12:06:48
2,ALFKI,24068,2012-07-19 12:06:48,2012-08-05 13:18:53
3,ALFKI,11181,2012-08-05 13:18:53,2012-08-06 02:27:56
4,ALFKI,18993,2012-08-06 02:27:56,2012-08-11 20:19:16
...,...,...,...,...
16813,WOLZA,24809,2016-01-07 18:06:15,2016-01-11 23:26:29
16814,WOLZA,21074,2016-01-11 23:26:29,2016-01-15 01:27:35
16815,WOLZA,14449,2016-01-15 01:27:35,2016-01-23 21:48:14
16816,WOLZA,15391,2016-01-23 21:48:14,2016-01-24 00:30:42
